<a href="https://colab.research.google.com/github/victorcharpentier/Anapix/blob/main/Pr%C3%A9dicteurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neptune-client
!pip install neptune-neptune_pytorch_lightning

NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjMDhiZDBkMC1jMjhkLTQ0YjgtYmVlYS02Y2E0ZThmNDY3NjIifQ=="


!pip install pytorch-lightning
#!pip install psutil
#!pip install sklearn
#!pip install torchvision
!pip install -U git+https://github.com/albumentations-team/albumentations_experimental
#!pip install ipywidgets

import psutil
import ipywidgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve, auc

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, lr_scheduler, SGD

import os

import albumentations as A
from albumentations.pytorch import ToTensor #ToTensorV2, ToTensor

from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning import loggers
from pytorch_lightning.metrics.functional import accuracy, auroc, recall, mean_squared_error



from collections import OrderedDict

     |████████████████████████████████| 245kB 9.1MB/s 
     |████████████████████████████████| 829kB 35.9MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 163kB 50.8MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 133kB 47.9MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.9.17-py2.py3-none-any.whl size=420060 sha256=11cd03ecdcb10ec964adb62a890308a669d95973b8e26ec1a54d9680909fd5d1
  Stored in directory: /root/.cache/pip/wheels/7d/f0/4f/3744064f7b502b302d303aca22dff170590ffd09020f58ba5f
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=92b044073e9b7a722802801d9961ec826f3d28306667132acec3e5804ad2e504
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp37-none

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
images_path = '/content/gdrive/My Drive/Anapix/vpc_autocropped_224/'

In [ ]:
cd /content/gdrive/My Drive/Anapix/

/content/gdrive/My Drive/Anapix


In [ ]:
yanal_label=pd.read_csv('yanal_label.csv', sep=',')
yanal_label.drop(yanal_label[yanal_label['patient_id']==199].index, axis=0, inplace=True)
yanal_label.head()

,patient_id,patient,image,concensus,path
0,13,1,1,4,P13I1.JPG
1,13,1,2,3,P13I2.JPG
2,13,1,3,3,P13I3.JPG
3,13,1,4,3,P13I4.JPG
4,13,1,5,3,P13I5.JPG


In [ ]:
class NewDataset(Dataset):
    
    """ Cette classe a vocation a être utilisé par la suite dans la récupération des images pour la partie clustering
    images dataset
    paramètres: 
    - df: dataframe containing at least 2 columns:
                1re colonne avec le nom ou le chemin de chaque image, 2e colonne avec le diagnostic
    - root_dir_image (string): Directory with all the images.
    - transform (callable, optional): Optional albumentations transformation to be applied
                on the image. 
    - autocrop (boolean): default is False. automaticaly detect the lesion and crop a square bounding box.
        if True, executes before transform and preprocess.
    - preprocess (callable, optional): additional transformation using albumentation
    """

    def __init__(self, data=yanal_label,transform=None, root_dir_image=None, colors=False):
        self.data=data
        self.root_dir_image=root_dir_image
        
        self.x=data["path"]  
        self.y=data["concensus"]
            
        self.transform = transform
        self.colors = colors
    
    def __len__(self):
        return len(self.xy)
    
    def mycolors(self, image):
      diff = np.mean(image, axis=(0,1)) - np.mean(image)
      image2 = image-diff
      return image2

    def preprocessing(self, img_name):
      if not os.path.isfile(img_name):
            #img_name = img_name.split('.')[0] + '.JPG'
            if not os.path.isfile(img_name):
              print('not found', img_name)
              raise IOError(f'The following file was not found: {img_name}')

      image = cv2.imread(img_name)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
      if self.transform:
          image = self.transform(image=image)['image']

      if self.colors:
        image = self.mycolors(image)

      image = image/255
      
      image = ToTensor()(image=image)['image'].float()
    
      return image

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        label = self.y.iloc[idx]
            
        img_name = os.path.join(self.root_dir_image,
                                self.x.iloc[idx])
        
        image = self.preprocessing(img_name)
    
        return image, label

In [ ]:
train = pd.read_csv('train_consensus_yanal.csv')
train.drop(train[train['patient_id']==199].index, axis=0, inplace=True)
test = pd.read_csv('test_consensus_yanal.csv')
test.sample(10)

,patient_id,path1,path2,consensus
576,176,P176I12.JPG,P176I9.JPG,8
190,15,P15I9.JPG,P15I4.JPG,4
470,165,P165I7.JPG,P165I13.JPG,5
2301,51,P51I9.JPG,P51I4.JPG,6
2980,68,P68I11.JPG,P68I33.JPG,4
4239,68,P68I6.JPG,P68I9.JPG,7
1032,26,P26I0.JPG,P26I27.JPG,4
1430,26,P26I5.JPG,P26I21.JPG,1
2616,60,P60I20.JPG,P60I18.JPG,8
1212,26,P26I29.JPG,P26I5.JPG,2


In [ ]:
train.insert(3, 'consensus_norma', train['consensus']/9)
test.insert(3, 'consensus_norma', test['consensus']/9)
test.sample(10)

,patient_id,path1,path2,consensus_norma,consensus
2379,60,P60I11.JPG,P60I11.JPG,1.000000,9
130,15,P15I5.JPG,P15I12.JPG,0.111111,1
4005,68,P68I35.JPG,P68I9.JPG,0.222222,2
1983,51,P51I24.JPG,P51I5.JPG,0.444444,4
3663,68,P68I28.JPG,P68I16.JPG,0.333333,3
471,165,P165I7.JPG,P165I14.JPG,0.222222,2
2085,51,P51I28.JPG,P51I18.JPG,0.666667,6
1704,51,P51I16.JPG,P51I14.JPG,0.888889,8
1467,51,P51I0.JPG,P51I1.JPG,0.000000,0
290,165,P165I14.JPG,P165I13.JPG,0.333333,3


In [ ]:
tr_labels = np.where(train['consensus']>3, 0, 1)
te_labels = np.where(test['consensus']>3, 0, 1)
train.insert(3,'label', tr_labels)
test.insert(3, 'label', te_labels)
test.sample(10)

KeyError: ignored

In [ ]:
train.insert(3,'distance_label', 1-train['consensus_norma'])
test.insert(3, 'distance_label', 1-test['consensus_norma'])
test.sample(10)

,patient_id,path1,path2,distance_label,label,consensus_norma,consensus
4208,68,P68I6.JPG,P68I15.JPG,0.555556,0,0.444444,4
1907,51,P51I22.JPG,P51I14.JPG,0.222222,0,0.777778,7
1991,51,P51I25.JPG,P51I11.JPG,0.555556,0,0.444444,4
3157,68,P68I16.JPG,P68I17.JPG,0.888889,1,0.111111,1
3243,68,P68I18.JPG,P68I24.JPG,0.555556,0,0.444444,4
2379,60,P60I11.JPG,P60I11.JPG,0.000000,0,1.000000,9
840,176,P176I7.JPG,P176I7.JPG,0.000000,0,1.000000,9
4338,68,P68I9.JPG,P68I27.JPG,0.777778,1,0.222222,2
1662,51,P51I14.JPG,P51I3.JPG,0.777778,1,0.222222,2
4311,68,P68I8.JPG,P68I38.JPG,0.666667,1,0.333333,3


# Vérifions la performance de chaque expert par rapport au concensus :

In [ ]:
yanal = pd.read_csv('yanal_30_2.csv', sep=';')
for expert in ["E1","E2","E3","E4","E5","E6","E7","E8","E9"]:
  compteur=0
  for index in range (len(test["path1"])):

    patient_id = test["patient_id"][index]

    image1 = test["path1"][index]
    nbim_image1 = image1[image1.index("I")+1:image1.index(".")]

    image2 = test["path2"][index]
    nbim_image2 = image2[image2.index("I")+1:image2.index(".")]

    concensus = test["consensus"][index]/9 #on récupère le concensus normalisé

    if int(yanal[ (yanal['patient_id'] == patient_id) & (yanal['image'] == int(nbim_image1) )][expert])== int(yanal[ (yanal['patient_id'] == patient_id) & (yanal['image'] == int(nbim_image2) )][expert]):
      compteur = compteur + 1 - ((concensus*9 -1)/8)  #on enlève le biais
      #print(1 - ((concensus*9 -1)/8))
    else:
      compteur = compteur +((concensus*9 +1)/8)
      #print(((concensus*9 +1)/8))
  print(compteur/len(test["path1"]))



0.43068625200826255
0.4624741794812945
0.42024328666513655
0.4228827174661464
0.4366536607757631
0.44365389029148494
0.4924259811797108
0.4149644250631168
0.43826027082855173


In [ ]:
class similiDataset(Dataset):
    """images dataset
    paramètres: 
    - df: dataframe containing at least 2 columns:
                1re colonne avec le nom ou le chemin de chaque image, 2e colonne avec le diagnostic
    - root_dir (string): Directory with all the images.
    - transform (callable, optional): Optional albumentations transformation to be applied
                on the image. 
    - autocrop (boolean): default is False. automaticaly detect the lesion and crop a square bounding box.
        if True, executes before transform and preprocess.
    - preprocess (callable, optional): additional transformation using albumentation
    """

    def __init__(self, df, root_dir='', transform=None, colors=False):
        self.data_frame = df.copy()
        self.root_dir = root_dir
        self.transform = transform
        self.colors = colors
    
    def __len__(self):
        return len(self.data_frame)
    
    def mycolors(self, image):
      diff = np.mean(image, axis=(0,1)) - np.mean(image)
      image2 = image-diff
      return image2

    def preprocessing(self, img_name):
      if not os.path.isfile(img_name):
            img_name = img_name.split('.')[0] + '.JPG'
            if not os.path.isfile(img_name):
              print('not found', img_name)
              raise IOError(f'The following file was not found: {img_name}')

      image = cv2.imread(img_name)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      
      if self.transform:
          image = self.transform(image=image)['image']

      if self.colors:
        image = self.mycolors(image)

      image = image/255
      image = ToTensor()(image=image)['image'].float()
      
      return image

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name1 = os.path.join(self.root_dir,
                                self.data_frame.iloc[idx, 1])
        img_name2 = os.path.join(self.root_dir,
                                self.data_frame.iloc[idx, 2])

      
        image1 = self.preprocessing(img_name1)
        image2 = self.preprocessing(img_name2)
        
        label = self.data_frame.iloc[idx, 3]

        return image1, image2, label

In [ ]:
images_path = 'vpc_autocropped_224/'

In [ ]:
train_transform = A.Compose([A.RandomRotate90(p=1),
                             A.HorizontalFlip(p=1),
                             A.CLAHE(p=0.5, clip_limit=(1, 5), tile_grid_size=(8, 8)),
                             A.GridDistortion(p=0.3, num_steps=5, distort_limit=(-0.3, 0.3), interpolation=0, border_mode=2),
                             A.augmentations.transforms.Cutout (num_holes=8, max_h_size=8, max_w_size=8, always_apply=False, p=0.2),
                             #A.RandomRain(always_apply=False, p=0.1, slant_lower=-5, slant_upper=5, drop_length=1, drop_width=1, drop_color=(0, 0, 0), blur_value=1, brightness_coefficient=0.8499999642372131, rain_type='drizzle'),
                             #A.augmentations.transforms.Normalize ( max_pixel_value=255.0, always_apply=True),
                             A.OneOf([
                                      #A.augmentations.transforms.ISONoise (color_shift=(0.01, 0.05), intensity=(0.1, 0.5), always_apply=False, p=1),
                                      A.augmentations.transforms.RandomBrightnessContrast (brightness_limit=0.2, contrast_limit=0.2, always_apply=False, p=1),
                                      A.MotionBlur(blur_limit=3, p=1),
                                      A.augmentations.transforms.Blur (blur_limit=3, always_apply=True),
                                      #A.MedianBlur(blur_limit=5),
                                      #A.GaussianBlur(blur_limit=5,sigma_limit=0),
                                      A.GaussNoise(var_limit=(5.0, 50.0)),
                                      ], p=0.7),
                             ])

In [ ]:
class ResnetMulti9cl(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.batch_size = hparams['batch_size']
        self.lr = hparams['lr']
        self.num_class = hparams['num_class']
        self.workers = hparams['workers']
        self.test_results = []
        self.test_labels = []
        
        self.model = models.resnet50(pretrained=True)
        num_features = self.model.fc.in_features
        #self.model.fc = nn.Sequential(nn.Linear(num_features, 1000),
        #                              nn.Linear(1000, self.num_class))
        self.model.fc = nn.Linear(num_features, self.num_class)
        for name, child in self.model.named_children():
           if name in ['layer3', 'layer4', 'avgpool', 'fc']:
               #print(name + ' is unfrozen')
               for param in child.parameters():
                   param.requires_grad = True
           else:
               #print(name + ' is frozen')
               for param in child.parameters():
                   param.requires_grad = False
        
    def forward(self, x):
        x = self.model(x)
        return x
ckpt_path_198 = 'epoch=11.ckpt'
model_198 = ResnetMulti9cl.load_from_checkpoint(checkpoint_path=ckpt_path_198)

In [ ]:
print(model)

In [ ]:
class Similitude(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
    
        self.batch_size = hparams['batch_size']
        self.lr = hparams['lr']
        self.workers = hparams['workers']
        
        self.test_results = []
        self.test_labels = []
        
        #self.model = models.mobilenet_v2(pretrained=True)
        #num_features = self.model.classifier[1].in_features
        #self.model.classifier[1] = nn.Linear(num_features, self.num_class)
        
        self.model = model_198.model
        #for params in self.model.parameters():
        #  params.requires_grad=False
        self.num_features = self.model.fc.in_features
        self.model.fc = torch.nn.Linear(self.num_features, 512)
        
        for name, child in self.model.named_children():
           if name in ['layer4', 'avgpool', 'fc']:
               #print(name + ' is unfrozen')
               for param in child.parameters():
                   param.requires_grad = True
           else:
               #print(name + ' is frozen')
               for param in child.parameters():
                   param.requires_grad = False

    def forward_once(self, x):
      output = self.model(x)
      return output
    
    def forward(self, input1, input2):
      output1 = self.forward_once(input1)
      output2 = self.forward_once(input2)
      return output1, output2
    
    def contrastive_loss(self, euclidean_distance, label):
      # perform contrastive loss calculation with the distance
      loss_contrastive = torch.mean(
          (1-label) * torch.pow(euclidean_distance, 2) +
          (label) * torch.pow(torch.clamp(1 - euclidean_distance, min=0.0), 2)
          )
      return loss_contrastive 

    def l1(self, logit, label):
      return nn.functional.l1_loss(logit.float(), label.float())

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.lr)
        #optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        #optimizer = torch.optim.SparseAdam(self.parameters(), lr=self.lr)
        #optimizer = torch.optim.ASGD(self.parameters(), lr=self.lr)
        #optimizer = torch.optim.RMSprop(self.parameters(), lr=self.lr)
        #optimizer = SGD(self.parameters(), lr=self.lr)
        #optimizer = torch.optim.Adagrad(self.parameters(), lr=self.lr, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                   patience=1, threshold=1e-3, verbose=True)    #https://pytorch.org/docs/stable/optim.html
        multistep = lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[4,12], gamma=0.1, verbose=True)
        scheduler_dict = {
                      'scheduler': scheduler,
                      'interval': 'epoch',
                      'monitor': 'val_loss',
                      'frequency': 1
                  }
        return [optimizer], [scheduler_dict]
    
    def training_step(self, train_batch, batch_id):
        x1, x2, y = train_batch
        logits1, logits2 = self.forward(x1, x2)
        # Find the pairwise distance or eucledian distance of two output feature vectors
        euclidean_distance = F.pairwise_distance(logits1, logits2)
        #print('distance:', euclidean_distance)
        #logits = logits.squeeze()
        #loss = self.contrastive_loss(euclidean_distance, y)
        loss = self.l1(euclidean_distance, y)

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)

        return {'loss': loss, 
                'label':y,
                'distance':euclidean_distance
                }

    def training_epoch_end(self, output):
        train_distance = torch.cat([x['distance'] for x in output])
        labels = torch.cat([x['label'] for x in output])
        mae = F.l1_loss(train_distance, labels)
        #roc = auroc(train_proba[:,1], labels)
        
        self.log('train_mae', mae)
    
    def validation_step(self, valid_batch, batch_id):
        x1, x2, y = valid_batch
        logits1, logits2 = self.forward(x1, x2)
        #logits = logits.squeeze()
        euclidean_distance = F.pairwise_distance(logits1, logits2)
        #loss = self.contrastive_loss(euclidean_distance, y)
        loss = self.l1(euclidean_distance, y)

        return {'loss': loss,
                'label':y,
                'distance':euclidean_distance
                }

    
    def validation_epoch_end(self, output):
        avg_loss = torch.stack([x['loss'] for x in output]).mean()
        val_distance = torch.cat([x['distance'] for x in output])
        labels = torch.cat([x['label'] for x in output])        
        mae = F.l1_loss(val_distance, labels)

        #tensorboard_logs = {'val_loss': avg_loss, 'valid_roc': roc}
        #for key, val in tensorboard_logs.items():
        #  self.log(key, val)
        self.log('val_loss', avg_loss)
        self.log('val_mae', mae)

    def test_step(self, batch, batch_idx):
        x, y, y_bin = batch
        logits = self.forward(x)
        #probas = nn.functional.softmax(logits, dim=1)
        #print('shape', probas.shape)
        #print('probas', probas[0])
        return logits
        #self.log('test_proba', probas)


    def test_step_end(self, output_results):
        #all_test_step_outs = output_results.out
        probas = nn.functional.softmax(output_results, dim=1)
        #loss = nce_loss(all_test_step_outs)
        print('all',probas.shape)
        #full_probas = torch.cat(output_results)
        tensorboard_logs = {'test_proba': output_results}


    def train_dataloader(self):
        return DataLoader(train_data, batch_size=self.batch_size, num_workers=self.workers, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(valid_data, batch_size=self.batch_size, num_workers=self.workers)

In [ ]:
train_data = similiDataset(train, images_path, transform=train_transform)
valid_data = similiDataset(test, images_path, transform=None)
hparams = {'batch_size': 30,
        'lr':1e-6,
        'workers': 2
        }


model = Similitude(hparams)

nep_logger = loggers.NeptuneLogger(api_key=NEPTUNE_API_TOKEN,
                                  project_name='charpentier.victor2/tests-similitudes',
                                params=hparams,
                                experiment_name='siamese',
                               tags=['optimizer_ADAM','lr_scheduler.ReduceLROnPlateau','l1_loss', 'new_transform', 'pretrained_198']
                               )

trainer = Trainer(gpus=1,logger = nep_logger,
                  max_epochs=15,
                  fast_dev_run=False)


trainer.fit(model)

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()